In [1]:
import random
import weave
weave.use_frontend_devmode()
from rich import print
from weave.ecosystem import scenario

#weave.panels.Each.WeaveType().to_dict()
#scenario.metrics_bank.to_dict()

In [2]:
# a = weave.save(5)
# b = a + 2
# weave.type_of(b)

In [3]:
import math
def make_run(n_scenarios, n) -> list[scenario.ScenarioResult]:
    return [
        scenario.ScenarioResult(
            scenario_id=str(i),
            metric1=random.expovariate(n / 33),
            metric2=random.gauss(42, n / 100),
            metric3=random.triangular(n, 11),
            metric4=random.random() * random.random() * n,
            metric5=random.random(),
            metric6=random.betavariate(0.01, n / 2),
        )
        for i in range(n_scenarios)
    ]

In [4]:
#panel_input = weave.save({'baseline': make_run(10), 'candidate': make_run(10)}, name='panel_input')

In [5]:
#panel_input

In [6]:
# make some runs

weave.save(make_run(100, 1), name='run1')
weave.save(make_run(100, 2), name='run2')
weave.save(make_run(100, 3), name='run3')
weave.save(make_run(100, 4), name='run4')
weave.save(make_run(100, 5), name='run5')
pass

In [7]:
# Ugly. We need to compute the Weave type right now.
# TODO: Fix
from weave import infer_types
run_type = weave.types.List(infer_types.python_type_to_type(scenario.ScenarioResult))

import time
runs = weave.ops.objects(run_type, 'latest', int(time.time()))

In [9]:
panel = weave.panels.Group2(
    preferHorizontal=True,
    items={
        "sidebar": weave.panels.Group2(
            style="width: 200px; padding: 16px;",
            items={
                "baseline": weave.panels.ObjectPicker(runs, label='baseline'),
                "candidate": weave.panels.ObjectPicker(runs, label='candidate')
            }
        ),
        "main": lambda sidebar: weave.panels.Group2(
            items={
                "tables": weave.panels.Group2(
                    style="height: 300px;",
                    preferHorizontal=True,
                    equalSize=True,
                    items={
                        "baseline_table": weave.panels.LabeledItem(
                            label="Baseline Table",
                            item=sidebar.config.items['baseline'].config.choice.get()
                        ),
                        "candidate_table": weave.panels.LabeledItem(
                            label="Candidate Table",
                            item=sidebar.config.items['candidate'].config.choice.get()
                        )
                    }),
                "plots": 
                    weave.panels.LabeledItem(
                        label='Scenario plots',
                        item=scenario.metrics_bank(
                            weave.ops.dict_(
                                baseline=sidebar.config.items['baseline'].config.choice.get(),
                                candidate=sidebar.config.items['candidate'].config.choice.get()
                            ))
                    )

            }
        )
    })
ref = weave.save(panel, name='my-new-panel')
ref